This notebook was used to generate the golden data results for ECG. It requires that the python-igraph package be installed to run. 

In [ ]:
from scipy.io import mmread
import networkx as nx
#mmFile='/datasets/kron_g500-logn21/kron_g500-logn21.mtx'
mmFile='/datasets/golden_data/graphs/dblp.mtx'
#mmFile='/datasets/networks/karate.mtx'
#mmFile='/home/jwyles/code/mycugraph/datasets/dolphins.mtx'
#mmFile='/home/jwyles/code/mycugraph/datasets/netscience.mtx'
M = mmread(mmFile).asfptype()
import cugraph
import cudf
import numpy as np
rows = cudf.Series(M.row)
cols = cudf.Series(M.col)
values = cudf.Series(M.data)
G = cugraph.Graph()
G.add_edge_list(rows, cols, values)

In [46]:
%%time
parts = cugraph.ecg(G, .05, 16)

CPU times: user 326 ms, sys: 400 ms, total: 726 ms
Wall time: 796 ms


In [47]:
numParts = parts['partition'].max() + 1
numParts

49204

In [48]:
mod = cugraph.analyzeClustering_modularity(G, numParts, parts['partition'])
mod

0.850147008895874

In [49]:
parts2, mod2 = cugraph.louvain(G)
mod2

0.7506256512679915

In [50]:
import igraph as ig
import numpy as np

def community_ecg(self, weights=None, ens_size=16, min_weight=0.05):
    W = [0]*self.ecount()
    ## Ensemble of level-1 Louvain 
    for i in range(ens_size):
        p = np.random.permutation(self.vcount()).tolist()
        g = self.permute_vertices(p)
        l = g.community_multilevel(weights=weights, return_levels=True)[0].membership
        b = [l[p[x.tuple[0]]]==l[p[x.tuple[1]]] for x in self.es]
        W = [W[i]+b[i] for i in range(len(W))]
    W = [min_weight + (1-min_weight)*W[i]/ens_size for i in range(len(W))]
    part = self.community_multilevel(weights=W)
    ## Force min_weight outside 2-core
    core = self.shell_index()
    ecore = [min(core[x.tuple[0]],core[x.tuple[1]]) for x in self.es]
    part.W = [W[i] if ecore[i]>1 else min_weight for i in range(len(ecore))]
    return part

ig.Graph.community_ecg = community_ecg

In [52]:
Gi = ig.Graph.Read_Edgelist('./dblp2.txt', directed=False)

In [53]:
%%time
ec = Gi.community_ecg()

CPU times: user 3min 49s, sys: 1.67 s, total: 3min 51s
Wall time: 3min 50s


In [36]:
ecg = np.zeros(len(Gi.vs), dtype=np.int32)

In [37]:
ecg

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [38]:
for i in range(len(ec)):
    for j in ec[i]:
        ecg[j] = i

In [39]:
ecg

array([275, 275,   0, ..., 435, 435, 107], dtype=int32)

In [40]:
ecg_col = cudf.Series(ecg)

In [41]:
numParts = ecg_col.max() + 1

In [42]:
mod4 = cugraph.analyzeClustering_modularity(G, numParts, ecg_col)

In [43]:
mod4

0.9279554486274719

In [94]:
maxId = 0
for i in range(len(ec)):
    for j in ec[i]:
        if j > maxId:
            maxId = j
maxId

34

In [79]:
len(Gi.es)

156

In [80]:
len(M.row)

156

In [81]:
78 *2

156

In [51]:
filename = "dblp2.txt"
f = open(filename, 'w')
for i in range(len(M.row)):
    f.write(str(M.row[i]) + ' ' + str(M.col[i]) + '\n')
f.close()

In [82]:
Gi.es[1]

igraph.Edge(<igraph.Graph object at 0x7fb7f439cd68>, 1, {})

In [84]:
Gi.es.select()[1]

igraph.Edge(<igraph.Graph object at 0x7fb7f439cd68>, 1, {})

In [85]:
Gi.es[0].source

0

In [88]:
Gi.es[0].target

1